## <font color='green'>Adaptação Projeto 1</font>
## <font color='green'>Vetores e Espaço Vetorial em Sistemas de Recomendação</font>

### Load package

In [1]:
# Imports
import ast
import nltk
import sklearn
import re
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None

## Carregando e Compreendendo os Dados

> https://developer.themoviedb.org/docs/getting-started

In [2]:
# Dataset de filmes
df_filmes = pd.read_csv("data/dataset_filmes.csv")

In [3]:
# Shape
df_filmes.shape

(4803, 20)

In [4]:
# Amostra dos dados
df_filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
# Dataset de elenco dos filmes
df_elenco = pd.read_csv("data/dataset_elenco.csv")

In [6]:
# Shape
df_elenco.shape

(4803, 4)

In [7]:
# Amostra dos dados
df_elenco.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Organização dos Dados de Texto

In [8]:
# Merge dos dataframes
df_filmes = df_filmes.merge(df_elenco, on = 'title')

In [9]:
# Shape
df_filmes.shape

(4809, 23)

In [10]:
# Discrepância no merge
df_filmes[df_filmes.id.duplicated()].count()

budget                  6
genres                  6
homepage                1
id                      6
keywords                6
original_language       6
original_title          6
overview                6
popularity              6
production_companies    6
production_countries    6
release_date            6
revenue                 6
runtime                 6
spoken_languages        6
status                  6
tagline                 6
title                   6
vote_average            6
vote_count              6
movie_id                6
cast                    6
crew                    6
dtype: int64

In [11]:
# Amostra dos dados
df_filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
# Info
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

> Temos diversos atributos que parecem ser irrelevantes para o projeto proposto.  
> Vou filtrar e manter os atributos categóricos com maior influência na definição da escolha. 

> Importante: a escolha dos atributos será feita conforme meu entendimento empírico sobre relevância na escolha do filme.

In [13]:
# Filtrando o dataframe e mantendo somente as colunas consideradas mais relevantes
df_filmes = df_filmes[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [14]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [15]:
# Duplicados depois do filter
df_filmes[df_filmes.title.duplicated()]

,movie_id,title,overview,genres,keywords,cast,crew
973,1255,The Host,A parasitic alien soul is injected into the bo...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 3, ""character"": ""Park Gang-du"", ""...","[{""credit_id"": ""52fe42eac3a36847f802ca6b"", ""de..."
1361,2661,Batman,The Dark Knight of Gotham City begins his war ...,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...","[{""id"": 848, ""name"": ""double life""}, {""id"": 84...","[{""cast_id"": 17, ""character"": ""Batman / Bruce ...","[{""credit_id"": ""52fe4363c3a36847f80509a7"", ""de..."
2879,72710,The Host,Gang-du is a dim-witted man working at his fat...,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 18, ""nam...","[{""id"": 1261, ""name"": ""river""}, {""id"": 1880, ""...","[{""cast_id"": 52, ""character"": ""Melanie Stryder...","[{""credit_id"": ""52fe487bc3a368484e0fa919"", ""de..."
2880,1255,The Host,Gang-du is a dim-witted man working at his fat...,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 18, ""nam...","[{""id"": 1261, ""name"": ""river""}, {""id"": 1880, ""...","[{""cast_id"": 3, ""character"": ""Park Gang-du"", ""...","[{""credit_id"": ""52fe42eac3a36847f802ca6b"", ""de..."
3651,10844,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 4470, ""name"": ""punk""}, {""id"": 10183, ""...","[{""cast_id"": 12, ""character"": ""Nick Harvey"", ""...","[{""credit_id"": ""52fe43c19251416c7501cceb"", ""de..."
3697,39269,Out of the Blue,Ordinary people find extraordinary courage in ...,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 2658, ""name"": ""new zealand""}, {""id"": 3...","[{""cast_id"": 2, ""character"": ""Don"", ""credit_id...","[{""credit_id"": ""52fe47099251416c9106826f"", ""de..."
3698,10844,Out of the Blue,Ordinary people find extraordinary courage in ...,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 2658, ""name"": ""new zealand""}, {""id"": 3...","[{""cast_id"": 12, ""character"": ""Nick Harvey"", ""...","[{""credit_id"": ""52fe43c19251416c7501cceb"", ""de..."
4272,268,Batman,The Dynamic Duo faces four super-villains who ...,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...","[{""id"": 339, ""name"": ""submarine""}, {""id"": 849,...","[{""cast_id"": 5, ""character"": ""Jack Napier/The ...","[{""credit_id"": ""52fe422fc3a36847f800aa4b"", ""de..."
4273,2661,Batman,The Dynamic Duo faces four super-villains who ...,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...","[{""id"": 339, ""name"": ""submarine""}, {""id"": 849,...","[{""cast_id"": 17, ""character"": ""Batman / Bruce ...","[{""credit_id"": ""52fe4363c3a36847f80509a7"", ""de..."


## Análise Exploratória e Limpeza dos Dados

In [16]:
# Shape
df_filmes.shape

(4809, 7)

In [17]:
# Checando valores ausentes
df_filmes.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [18]:
# Onde estão os valores ausentes
df_filmes[df_filmes.isnull().any(axis=1)]

,movie_id,title,overview,genres,keywords,cast,crew
2658,370980,Chiamatemi Francesco - Il Papa della gente,NaN,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...","[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4145,459488,"To Be Frank, Sinatra at 100",NaN,"[{""id"": 99, ""name"": ""Documentary""}]","[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...","[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4437,292539,Food Chains,NaN,"[{""id"": 99, ""name"": ""Documentary""}]",[],[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [19]:
# Temos somente 3 valores ausentes, considerando o genero dos filmes, 
# a quantidade de ausentes e a importância do "overview" na escolha de um filme,
# vou optar por remover os ausentes. 
df_filmes.dropna(inplace = True)

In [20]:
# Checando valores ausentes
df_filmes.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [21]:
# Checando se há valores duplicados
df_filmes.duplicated().sum()

np.int64(0)

In [22]:
# Shape
df_filmes.shape

(4806, 7)

> Continuamos com valores discrepantes no merge, vou resolver depois do processamento

## Processamento de Texto com Abstract Syntax Trees

https://docs.python.org/3/library/ast.html

Vamos começar ajustando a coluna de gênero de filme. Nesta coluna precisamos apenas dos nomes e não dos ids.

In [23]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


https://docs.python.org/3/library/ast.html

O módulo ast ajuda a descobrir programaticamente como é a gramática atual de um objeto, processando árvores da gramática de sintaxe abstrata.

In [24]:
def converter(obj):
    """
    Usamos ast.literal_eval(obj) para avaliar/converter a string obj em uma estrutura de dados Python.
    """
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [25]:
# Aplicamos então a função ao nosso dataset, na coluna de gênero do filme
df_filmes['genres'] = df_filmes['genres'].apply(converter)

In [26]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [27]:
# Aplicamos então a função ao nosso dataset, na coluna de keywords do filme
df_filmes['keywords'] = df_filmes['keywords'].apply(converter)

In [28]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Extrair membros do elenco.

In [29]:
# Função de conversão
def convert_member(cast, qty=5):
    
    # Cria uma lista vazia chamada L para armazenar os nomes.
    L = []  
    
    # Inicializa um contador para controlar o número de elementos adicionados à lista.
    counter = 0  
    
    # Usa ast.literal_eval para avaliar a string 'obj' e iterar sobre os elementos resultantes.
    for i in ast.literal_eval(cast):  
        
        # Verifica se o contador é diferente.
        if counter != qty:  
            
            # Adiciona o valor da chave 'name' do dicionário atual à lista L.
            L.append(i['name'])  
            
            # Incrementa o contador.
            counter += 1  
        else:
            
            # Interrompe o loop.
            break  
    
    # Retorna a lista L.
    return L  

In [30]:
# Aplicamos então a função ao nosso dataset, na coluna de cast (elenco) do filme
df_filmes['cast'] = df_filmes['cast'].apply(convert_member)

In [31]:
df_filmes.tail()

,movie_id,title,overview,genres,keywords,cast,crew
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]",[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."
4808,25975,My Date with Drew,Ever since the second grade when he first saw ...,[Documentary],"[obsession, camcorder, crush, dream girl]","[Drew Barrymore, Brian Herzlinger, Corey Feldm...","[{""credit_id"": ""58ce021b9251415a390165d9"", ""de..."


A função abaixo vai extrair o nome do diretor do filme.

In [32]:
# Função de conversão
def fetch_director(obj):
    
    # Cria uma lista vazia chamada L para armazenar o nome do diretor.
    L = []
    
    # Converte a string 'obj' em um objeto Python e itera sobre seus elementos.
    for i in ast.literal_eval(obj):
        
        # Verifica se o trabalho ('job') do elemento atual é 'Director'.
        if i['job'] == 'Director':
            
            # Adiciona o nome do diretor (valor da chave 'name') à lista L.
            L.append(i['name'])
            
            # Interrompe o loop após encontrar o primeiro diretor.
            break
            
    # Retorna a lista L, contendo o nome do diretor.
    return L

In [33]:
# Aplicamos então a função ao nosso dataset, na coluna de crew (equipe) do filme
df_filmes['crew'] = df_filmes['crew'].apply(fetch_director)

In [34]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [35]:
df_filmes[df_filmes.title.duplicated()]

,movie_id,title,overview,genres,keywords,cast,crew
973,1255,The Host,A parasitic alien soul is injected into the bo...,"[Action, Adventure, Romance, Science Fiction, ...","[based on novel, mass murder, dystopia, genoci...","[Song Kang-ho, Park Hae-il, Bae Doona, Ko Ah-s...",[Bong Joon-ho]
1361,2661,Batman,The Dark Knight of Gotham City begins his war ...,"[Fantasy, Action]","[double life, dc comics, dual identity, chemic...","[Adam West, Burt Ward, Lee Meriwether, Cesar R...",[Leslie H. Martinson]
2879,72710,The Host,Gang-du is a dim-witted man working at his fat...,"[Horror, Drama, Science Fiction]","[river, mobile phone, bravery, archer, daughte...","[Saoirse Ronan, Diane Kruger, Jake Abel, Willi...",[Andrew Niccol]
2880,1255,The Host,Gang-du is a dim-witted man working at his fat...,"[Horror, Drama, Science Fiction]","[river, mobile phone, bravery, archer, daughte...","[Song Kang-ho, Park Hae-il, Bae Doona, Ko Ah-s...",[Bong Joon-ho]
3651,10844,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,[Drama],"[punk, independent film, coming of age, teenag...","[Karl Urban, Tandi Wright, Simon Ferry, Matthe...",[Robert Sarkies]
3697,39269,Out of the Blue,Ordinary people find extraordinary courage in ...,[Drama],"[new zealand, gun rampage, police, independent...","[Dennis Hopper, Raymond Burr, Sharon Farrell, ...",[Dennis Hopper]
3698,10844,Out of the Blue,Ordinary people find extraordinary courage in ...,[Drama],"[new zealand, gun rampage, police, independent...","[Karl Urban, Tandi Wright, Simon Ferry, Matthe...",[Robert Sarkies]
4272,268,Batman,The Dynamic Duo faces four super-villains who ...,"[Family, Adventure, Comedy, Science Fiction, C...","[submarine, dc comics, missile, shark attack, ...","[Jack Nicholson, Michael Keaton, Kim Basinger,...",[Tim Burton]
4273,2661,Batman,The Dynamic Duo faces four super-villains who ...,"[Family, Adventure, Comedy, Science Fiction, C...","[submarine, dc comics, missile, shark attack, ...","[Adam West, Burt Ward, Lee Meriwether, Cesar R...",[Leslie H. Martinson]


In [36]:
# Removo duplicatas dos filmes com mesmo movie_id
df_filmes.drop_duplicates(subset='movie_id', inplace=True)

In [37]:
df_filmes.shape

(4800, 7)

## Limpeza dos Dados de Texto

In [38]:
# Separamos a string da coluna overview por espaço em branco
df_filmes['overview'] = df_filmes['overview'].apply(lambda x:x.split())

In [39]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


## Preparando os Dados Para Vetorização em Outro Espaço Vetorial

In [40]:
# Criamos a coluna de tags, nesse caso um vetor de strings com os valores das colunas
df_filmes['tags'] = df_filmes['overview'] + \
                        df_filmes['genres'] + \
                        df_filmes['keywords'] + \
                        df_filmes['cast'] + \
                        df_filmes['crew']

In [41]:
df_filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [42]:
# Dataset final
df_filmes_novo = df_filmes[['movie_id', 'title', 'tags']]

In [43]:
df_filmes_novo.head(10)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
5,559,Spider-Man 3,"[The, seemingly, invincible, Spider-Man, goes,..."
6,38757,Tangled,"[When, the, kingdom's, most, wanted-and, most,..."
7,99861,Avengers: Age of Ultron,"[When, Tony, Stark, tries, to, jumpstart, a, d..."
8,767,Harry Potter and the Half-Blood Prince,"[As, Harry, begins, his, sixth, year, at, Hogw..."
9,209112,Batman v Superman: Dawn of Justice,"[Fearing, the, actions, of, a, god-like, Super..."


In [44]:
# Join das strings para simplificar o vetor
df_filmes_novo['tags'] = df_filmes_novo['tags'].apply(lambda x:" ".join(x))

In [45]:
def remove_punct(text):
    """
    Remove punctuation and return stripped sentences and lower characters"""
    return re.sub(r'[^\w\s]', '', text).lower().strip()

In [46]:
# Coloca tudo em minúsculo para evitar diferença de palavras maiúsculo/minúsculo
df_filmes_novo['tags'] = df_filmes_novo['tags'].apply(remove_punct)

# faço o mesmo para os títulos
df_filmes_novo['title'] = df_filmes_novo.title.apply(remove_punct)

In [47]:
df_filmes_novo.head(10)

,movie_id,title,tags
0,19995,avatar,in the 22nd century a paraplegic marine is dis...
1,285,pirates of the caribbean at worlds end,captain barbossa long believed to be dead has ...
2,206647,spectre,a cryptic message from bonds past sends him on...
3,49026,the dark knight rises,following the death of district attorney harve...
4,49529,john carter,john carter is a warweary former military capt...
5,559,spiderman 3,the seemingly invincible spiderman goes up aga...
6,38757,tangled,when the kingdoms most wantedand most charming...
7,99861,avengers age of ultron,when tony stark tries to jumpstart a dormant p...
8,767,harry potter and the halfblood prince,as harry begins his sixth year at hogwarts he ...
9,209112,batman v superman dawn of justice,fearing the actions of a godlike super hero le...


## Parse e Vetorização

In [48]:
# parser
parser_ps = PorterStemmer()

In [49]:
# Função de stemming
def stem(text):
    
    # Cria uma lista vazia chamada y para armazenar as palavras após o stemming.
    y = []
    
    # Divide a string de entrada 'text' em palavras e itera sobre elas.
    for i in text.split():
        
        # Realiza o stemming na palavra atual 'i' e adiciona o resultado à lista y.
        y.append(parser_ps.stem(i))
    
    # Retorna as palavras processadas como uma string, unindo-as com espaços.
    return " ".join(y)

In [50]:
# Aplica a função à coluna de tags
df_filmes_novo['tags'] = df_filmes_novo['tags'].apply(stem)

In [51]:
df_filmes_novo.head()

,movie_id,title,tags
0,19995,avatar,in the 22nd centuri a parapleg marin is dispat...
1,285,pirates of the caribbean at worlds end,captain barbossa long believ to be dead ha com...
2,206647,spectre,a cryptic messag from bond past send him on a ...
3,49026,the dark knight rises,follow the death of district attorney harvey d...
4,49529,john carter,john carter is a warweari former militari capt...


In [52]:
# Cria o vetorizador com no máximo 5000 atributos
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [53]:
# Cria os vetores para as tags
vectors = cv.fit_transform(df_filmes_novo['tags']).toarray()

In [54]:
cv.get_feature_names_out()[0:200]

array(['007', '10', '100', '10yearold', '11', '12', '12yearold', '13',
       '15', '16', '16th', '16yearold', '17', '17th', '17yearold', '18',
       '18th', '1910', '1920', '1930', '1940', '1950', '1960', '1970',
       '1980', '1985', '1990', '19th', '20', '2000', '2009', '20th',
       '21st', '25', '30', '3d', '40', '50', '51', '60', '70', '911',
       'aaron', 'abandon', 'abbi', 'abduct', 'abel', 'abigail', 'abil',
       'abl', 'aboard', 'aborigin', 'abov', 'abraham', 'abroad', 'abus',
       'academi', 'accent', 'accept', 'access', 'accid', 'accident',
       'acclaim', 'accompani', 'accomplish', 'account', 'accus', 'ace',
       'achiev', 'acquaint', 'act', 'action', 'actionpack', 'activ',
       'activist', 'actor', 'actress', 'actual', 'ad', 'adam', 'adapt',
       'add', 'addict', 'adjust', 'admir', 'admit', 'adolesc', 'adopt',
       'ador', 'adrian', 'adrien', 'adult', 'adulteri', 'adulthood',
       'advanc', 'advantag', 'adventur', 'advertis', 'advic', 'advis',
       

In [55]:
# Para visualizar todas as colunas do array
np.set_printoptions(threshold = np.inf)

In [56]:
len(vectors[0])

5000

## Cálculo de Distância dos Vetores

Com os vetores usamos o cálculo da distância de cosseno (cosine_similarity), este nos indica o quanto os vetores estão próximos.  
Que por sua vez indica a similaridade (quanto mais próximos mais similares).

In [57]:
# Calcula a similaridade entre os vetores calculando as distâncias entre eles
similaridades = cosine_similarity(vectors)

## Construindo o Sistema de Recomendação

In [70]:
# Função para o sistema de recomendação
def sistema_recomendacao(movie, qty=5):
    """
    movie: Nome do Filme ao qual deseja ver similares
    qty: Quantidade de indicações """
    
    # Obtém o índice do filme passado como argumento (o que o usuário assistiu)
    index = df_filmes_novo[df_filmes_novo['title'] == remove_punct(movie)].index[0]
    
    # Verificamos então os filmes com vetores de menor distância para o filme passado como argumento
    distances = sorted(list(enumerate(similaridades[index])), reverse = True, key = lambda x: x[1])
    
    # E então consideramos os filmes com menor distância, ou seja, maior similaridade
    for i in distances[1: qty + 1]:
        print(df_filmes_novo.iloc[i[0]].title.title())

## Aplicando o Sistema de Recomendação

In [71]:
# Quais as recomendações de filmes para quem assistiu ao filme: Avengers: Age of Ultron?
sistema_recomendacao('Avengers: Age of Ultron')

The Avengers
Captain America Civil War
Iron Man 2
Iron Man
Iron Man 3


In [72]:
# Quais as recomendações de filmes para quem assistiu ao filme: Jurassic World?
sistema_recomendacao('Jurassic World', 10)

Jurassic Park
The Lost World Jurassic Park
Vacation
Jurassic Park Iii
The Nut Job
The Curse Of The Wererabbit
Spirited Away
The Beast From 20000 Fathoms
Walking With Dinosaurs
Broken Arrow


In [75]:
# Quais as recomendações de filmes para quem assistiu ao filme: Gladiator?
sistema_recomendacao('GLADIATOR', 20)

1982
After Earth
The Diving Bell And The Butterfly
Hercules
Flying By
Krrish
Copying Beethoven
Shouldve Been Romeo
Juliet And Alfa Romeo
Micmacs
The Beastmaster
Resurrecting The Champ
Big Fish
All Is Lost
Gran Torino
Pompeii
The Rm
The Legend Of Hercules
Homefront
The Barbarian Invasions


In [88]:
# Quais as recomendações de filmes para quem assistiu ao filme: The Hobbit: The Battle of the Five Armies?
sistema_recomendacao('The Hobbit: the BATTLE@ of the Five- ArmiEs', 3)

The Hobbit The Desolation Of Smaug
The Hobbit An Unexpected Journey
The Lord Of The Rings The Fellowship Of The Ring


> O objetivo do projeto foi atingido, que é mostrar um sistema de recomendação  
que pode ser ampliado para qualquer tipo de produto ou serviço.